# 전체 웹툰 댓글 가져오기

In [ ]:
import csv

f = open('webtoon_comment.csv','w')
wr = csv.writer(f)

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
def get_href(soup) :
    # 요일별 전체 웹툰 가져오기
    
    div = soup.find('div', class_ = 'list_area daily_all')
    result = []
    for a in div.find_all('li'):
        result.append(a.find('a')['href'])
        
    return result

def main():
    list_href = []

    url = "https://comic.naver.com/webtoon/weekday.nhn"
    result = requests.get(url)
    soup = BeautifulSoup(result.text, "html.parser")

    code = get_code(soup)
    code = list(code.keys())
    
    
    get_comment(code)
    

if __name__ == "__main__":
    main()

# 코드따오기

In [ ]:
def get_code(soup):
    title = soup.select('.title')
    t_IDs=list(map(lambda x: x.get('href').split('titleId=')[1].split('&')[0], title))
    t_weekdays = list(map(lambda x: x.get('href').split('weekday=')[1], title))
    t_names = list(map(lambda x: x.text ,title))

    #크롤링이 잘 되었나 확인하기 위함, 총 웹툰 수
    data ={}
    for i in range(len(t_IDs)):
        data[t_IDs[i]] =t_names[i]
        
        
    return(data)

# 댓글 가져오기  JSON

In [ ]:
  
def get_comment(code):
    headers = {
        'authority': 'apis.naver.com',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
        'accept': '*/*',
        'sec-fetch-site': 'same-site',
        'sec-fetch-mode': 'no-cors',
        'sec-fetch-dest': 'script',
        'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=748105&no=22',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'cookie': 'NRTK=ag#20s_gr#0_ma#-2_si#-2_en#-2_sp#-2; NID_AUT=uCu9XVwzszVbbRRyqhJfBcaf6ZmkaWn8c8j3HTnh5evGiJeQuCCd4KP4WOk8vJ0R; NID_JKL=Bx5L3SR5iqL0LSsQQELWuoLp6kVPZSrh42Uq2L054cM=; ASID=3a78904e00000173065ae2720000004a; MM_NEW=1; NFS=2; MM_NOW_COACH=1; NNB=2WB2COMNMUKV6; _ga=GA1.1.406437722.1593356627; _ga_4BKHBFKFK0=GS1.1.1597672842.2.1.1597672844.58; nx_ssl=2; page_uid=U0kIxlpzCx4sskB4+V4-490774; NID_SES=AAABi9+G8m4uMNubDAMMQMCc+Lh8eaoVk+1qg5RkpHvJ2REsN2cEYvOCClKHqKwkzBFZjYVsKmFNcptW/1ngQPbGYxckWL8RY0JSCMmOXbcXJxruC0QSCN5jXrkLLZ9WooSs1IAh3b+SdbgkpiAW9kvK2fQqL2vbYNT6rI0rCvtwwP7Usn7LwklDNPOE7CGsw3P1iFOglvx9I+NMYOXTuz4xqGTJr/kSxr0zx5o+/xzp+rIhILXexH+gLYAfp30k2YlgtvNuOcdQorLp64JDsQBDAZG41Fqj2Q8G1UqWbyv8pOjWYQKtXBfmtquh/JR+SYfQ1UPQ6kcM5QsMse4rd4MneK0ozGOeXy1CaJjoIp1HJyp4yfxFI1IsCEufI/Go2Yyv6tUDZ32O+89rHyL4Q2+aClR5U7eRJsBgJYzCS8hewtgaEb/KZbsGUZ18LQwq1YZN1dieIwTlYrySb+J15TPB4IgSeNhWJiwUqEnw+U/VEt676hsNVsoONEgtOnSggIx7af3oBkv/vXV7BiZFciz3LKY=; BMR=s=1598497364509&r=https%3A%2F%2Fm.blog.naver.com%2FPostView.nhn%3FblogId%3Dpjt3591oo%26logNo%3D220962168316%26proxyReferer%3Dhttps%3A%252F%252Fwww.google.com%252F&r2=https%3A%2F%2Fwww.google.com%2F',
        }

    for code in code:
        for i in range(1,web_num(code)+1):
            params = (
                ('ticket', 'comic'),
                ('templateId', 'webtoon'),
                ('pool', 'cbox3'),
                ('_callback', 'jQuery1124003662081780437343_1598497437731'),
                ('lang', 'ko'),
                ('country', 'KR'),
                ('objectId', str(code)+'_'+str(i)),
                ('categoryId', ''),
                ('pageSize', '15'),
                ('indexSize', '10'),
                ('groupId', ''),
                ('listType', 'OBJECT'),
                ('pageType', 'default'),
                ('page', '1'),
                ('initialize', 'true'),
                ('userType', ''),
                ('useAltSort', 'true'),
                ('replyPageSize', '10'),
                ('_', '1598497437732'),
            )

            response = requests.get('https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json', headers=headers, params=params)
            res = response.text 
            jdict = ''
            for i in range(len(res)) :
                if res[i] == '(' :
                    jdict = res[i + 1: -2]
                    break;
            jdict = json.loads(jdict)
        #     print(jdict)
            jdict = jdict['result']['commentList']

            for i in jdict :
                # return(code + ' ' + i['contents'])
                wr.writerow([code,[i['contents']]])

# 편수 확인

In [ ]:
def web_num(code):
    url = "https://comic.naver.com/webtoon/list.nhn?titleId="+str(code)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    num_tag = soup.select(' table.viewList > tr > td.title > a')


    t_num = list(map(lambda x: x.get('href').split('no=')[1].split('&')[0], num_tag))


    max = t_num[0]
    web_num = int(max)
    return web_num